## Set up:

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir
from os.path import join
import pandas as pd
import re
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
import contextily as cx
import matplotlib.colors as mcolors
from shapely.geometry import box
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
import massbalancemachine as mbm
from cmcrameri import cm

from scripts.geodata import *
from scripts.geodata_plots import *
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.xgb_helpers import *
#  Suppress warnings issued by Cartopy when downloading data files
warnings.filterwarnings('ignore')

# reload modules
%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

cmap = cm.devon
color_palette_glaciers = sns.color_palette(get_cmap_hex(cmap, 15))

colors = get_cmap_hex(cm.batlow, 2)
color_xgb = colors[0]
color_tim = '#c51b7d'

In [ ]:
vois_climate = [
    't2m_corr', 'tp_corr', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]
vois_topographical = [
    "aspect_sgi",
    "slope_sgi",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
]
# Feature columns:
feature_columns = [
    'ELEVATION_DIFFERENCE'
] + list(vois_climate) + list(vois_topographical) + ['pcsr']
all_columns = feature_columns + cfg.fieldsNotFeatures

In [ ]:
# RGI Ids:
# Read rgi ids:
path_rgi = '../../../data/GLAMOS/CH_glacier_ids_long.csv'
rgi_df = pd.read_csv(path_rgi, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)
rgi_df.head(2)

In [ ]:
# Load the shapefile
shapefile_path = "../../../data/GLAMOS/topo/SGI2020/SGI_2016_glaciers_copy.shp"
gdf_shapefiles = gpd.read_file(shapefile_path)

# Order glaciers by size:
gl_area = {}
for glacierName in rgi_df.index:
    if glacierName == 'clariden':
        sgi_id = rgi_df.loc['claridenL']['sgi-id'].strip()
        rgi_shp = rgi_df.loc['claridenL']['rgi_id_v6_2016_shp']
    else:
        sgi_id = rgi_df.loc[glacierName]['sgi-id'].strip()
        rgi_shp = rgi_df.loc[glacierName]['rgi_id_v6_2016_shp']

    # 2016 shapefile of glacier
    gdf_mask_gl = gdf_shapefiles[gdf_shapefiles.RGIId == rgi_shp]
    gl_area[glacierName] = gdf_mask_gl.Area.values[0]

gl_area['clariden'] = gl_area['claridenL']

# sort by area (smallest first)
glacier_list = sorted(gl_area, key=lambda x: gl_area[x], reverse=False)
glacier_list

In [ ]:
PATH_PREDICTIONS = 'results/nc/glamos/OGGM_vars/TP_corr_T2m_corr/'

# PATH_PREDICTIONS = 'results/nc/glamos/Simple_vars/TP_corr_T2m_corr/'

In [ ]:
path_glacier_grid_glamos

## Glaciers covered by GLAMOS:

In [ ]:
# read geodetic MB:
geodeticMB = pd.read_csv(path_geodetic_MB_glamos + 'dV_DOI2024_allcomb.csv')

# get rgi of those glaciers:
rgi_gl = []
for gl in os.listdir(PATH_PREDICTIONS):
    if gl == 'clariden':
        rgi_gl.append(rgi_df.loc['claridenU']['rgi_id_v6_2016_shp'])
    else:
        rgi_gl.append(rgi_df.loc[gl]['rgi_id_v6_2016_shp'])

sgi_gl = [
    rgi_df[rgi_df['rgi_id_v6_2016_shp'] == rgi]['sgi-id'].values[0]
    for rgi in rgi_gl
]
geodeticMB = geodeticMB[geodeticMB['SGI-ID'].isin(sgi_gl)]

# Add glacierName to geodeticMB
# based  on SGI-ID
glacierNames = [
    rgi_df[rgi_df['sgi-id'] == sgi_id].index[0]
    for sgi_id in geodeticMB['SGI-ID'].values
]
# replace claridenL by clariden
glacierNames = [
    glacierName.replace('claridenL', 'clariden')
    for glacierName in glacierNames
]
geodeticMB['glacierName'] = glacierNames

periods_per_glacier = defaultdict(list)
geoMB_per_glacier = defaultdict(list)
# Iterate through the DataFrame rows
for _, row in geodeticMB.iterrows():
    glacierName = row['glacierName']
    start_year = row['Astart']
    end_year = row['A_end']
    geoMB = row['Bgeod']

    # Append the (start, end) tuple to the glacier's list
    periods_per_glacier[glacierName].append((start_year, end_year))
    geoMB_per_glacier[glacierName].append(geoMB)

In [ ]:
# List the files in both directories
glaciers_in_distributed_glamos = os.listdir(path_distributed_MB_glamos)
glaciers_in_geodetic = periods_per_glacier.keys()
glaciers_in_glamos_corr = os.listdir(PATH_PREDICTIONS)

# Check if all glaciers in the first directory are in the second directory
missing_glaciers = [
    glacier for glacier in glaciers_in_distributed_glamos
    if glacier not in glaciers_in_glamos_corr
]
missing_glaciers_geod = [
    glacier for glacier in glaciers_in_geodetic
    if glacier not in glaciers_in_glamos_corr
]

# Find the intersection of the two lists
intersecting_glaciers = list(
    set(missing_glaciers_geod) & set(missing_glaciers))


# Sort the lists by area if available in gl_area
def sort_by_area(glacier_list):
    return sorted(glacier_list, key=lambda g: gl_area.get(g, 0), reverse=False)


missing_glaciers_sorted_gridded = sort_by_area(missing_glaciers)
missing_glaciers_geod_sorted = sort_by_area(missing_glaciers_geod)
intersecting_glaciers_sorted = sort_by_area(intersecting_glaciers)

print("The following glaciers are missing compared to gridded MB:")
print(missing_glaciers_sorted_gridded)

print("The following glaciers are missing compared to the geodetic MB:")
print(missing_glaciers_geod_sorted)

print("The following glaciers are missing in both lists:")
print(intersecting_glaciers_sorted)

In [ ]:
# intersection with geodetic MB:
glacier_list_geod = sort_by_area([
    glacier for glacier in glaciers_in_geodetic
    if glacier in glaciers_in_glamos_corr
])
glacier_list_geod

## Geodetic MB:

In [ ]:
def get_GLAMOS_glwmb(glacier_name):
    """
    Loads and processes GLAMOS glacier-wide mass balance data.

    Parameters:
    -----------
    glacier_name : str
        The name of the glacier.

    Returns:
    --------
    pd.DataFrame or None
        A DataFrame with columns ['YEAR', 'GLAMOS Balance'] indexed by 'YEAR',
        or None if the file is missing.
    """

    # Construct file path safely
    file_path = os.path.join(path_SMB_GLAMOS_csv, "fix",
                             f"{glacier_name}_fix.csv")

    # Check if file exists
    if not os.path.exists(file_path):
        print(
            f"Warning: GLAMOS data file not found for {glacier_name}. Skipping..."
        )
        return None

    # Load CSV and transform dates
    df = pd.read_csv(file_path)
    df = transformDates(df)

    # Remove duplicates based on the date column
    df = df.drop_duplicates(subset=["date1"])

    # Ensure required columns exist
    required_columns = {"date1", "Annual Balance"}
    if not required_columns.issubset(df.columns):
        print(
            f"Warning: Missing required columns in {glacier_name} GLAMOS data. Skipping..."
        )
        return None

    # Extract year from date and normalize balance
    df["YEAR"] = pd.to_datetime(df["date1"]).dt.year.astype("int64")
    df["GLAMOS Balance"] = df[
        "Annual Balance"] / 1000  # Convert to meters water equivalent

    # Select relevant columns and set index
    return df[["YEAR", "GLAMOS Balance"]].set_index("YEAR")

#### Example:

In [ ]:
glacier_name = "clariden"
GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name)

# If GLAMOS data is missing, skip processing
if GLAMOS_glwmb is None:
    print(f"Skipping {glacier_name}: No GLAMOS data available.")
else:
    # Get all geodetic periods
    periods = periods_per_glacier.get(glacier_name, [])
    geoMBs = geoMB_per_glacier.get(glacier_name, [])

    path_mbm_pred = os.path.join(PATH_PREDICTIONS, glacier_name)

    # Storage lists
    mbm_mb_mean, glamos_mb_mean, geodetic_mb, target_period = [], [], [], []

    for period in periods:
        mbm_mb, glamos_mb = [], []

        for year in range(period[0], period[1] + 1):
            # Construct file path
            file_path = os.path.join(path_mbm_pred,
                                     f"{glacier_name}_{year}_annual.nc")

            # Check if the NetCDF file exists
            if not os.path.exists(file_path):
                print(
                    f"Warning: Missing MBM file for {glacier_name} ({year}). Skipping..."
                )
                mbm_mb.append(np.nan)
            else:
                ds = xr.open_dataset(file_path)
                # Compute mean glacier-wide MB
                mbm_mb.append(ds["pred_masked"].mean().values)

            # Get GLAMOS Balance for the year, or return NaN if missing
            glamos_mb.append(GLAMOS_glwmb["GLAMOS Balance"].get(year, np.nan))

        # Store mean values, ignoring NaNs
        mbm_mb_mean.append(np.nanmean(mbm_mb))
        glamos_mb_mean.append(np.nanmean(glamos_mb))
        geodetic_mb.append(geoMBs[periods.index(period)])
        target_period.append(period)

fig = plotGeodeticMB(geodetic_mb, mbm_mb_mean, glamos_mb_mean, glacier_name,
                     color_xgb, color_tim)

#### All glaciers:

In [ ]:
def check_missing_years(folder_path, glacier_name, period):
    start_year, end_year = period
    expected_years = set(range(start_year, end_year + 1))

    # Extract years from filenames
    available_years = set()
    pattern = re.compile(rf'{glacier_name}_(\d{{4}})_annual\.nc')

    for filename in os.listdir(folder_path):
        match = pattern.match(filename)
        if match:
            year = int(match.group(1))
            available_years.add(year)

    missing_years = expected_years - available_years
    if missing_years:
        return True
    else:
        return False

In [ ]:
# Define path for saving figures
path_figure = "figures/geodetic/"
emptyfolder(path_figure)  # Ensure directory is empty before saving

for glacier_name in glacier_list_geod:

    # Check if GLAMOS file exists
    glamos_file = os.path.join(path_SMB_GLAMOS_csv, "fix",
                               f"{glacier_name}_fix.csv")
    if not os.path.exists(glamos_file):
        print(f"Skipping {glacier_name}: No GLAMOS data available.")
        continue

    # Load GLAMOS data
    GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name)
    if GLAMOS_glwmb is None:
        print(f"Skipping {glacier_name}: Failed to load GLAMOS data.")
        continue

    # Get all geodetic periods
    periods = periods_per_glacier.get(glacier_name, [])
    geoMBs = geoMB_per_glacier.get(glacier_name, [])

    if not periods or not geoMBs:
        print(
            f"Skipping {glacier_name}: No geodetic mass balance data available."
        )
        continue

    # Open glacier-wide MB nc files for all years of the period
    folder_path = os.path.join(PATH_PREDICTIONS, glacier_name)

    mbm_mb_mean, glamos_mb_mean, geodetic_mb, target_period = [], [], [], []

    for period in periods:
        # Skip if any required year is missing in the dataset
        if check_missing_years(folder_path, glacier_name, period):
            print(
                f"Skipping period {period} for {glacier_name} due to missing files."
            )
            continue

        mbm_mb, glamos_mb = [], []

        for year in range(period[0], period[1] + 1):
            file_path = os.path.join(folder_path,
                                     f"{glacier_name}_{year}_annual.nc")

            # Check if the NetCDF file exists before loading
            if not os.path.exists(file_path):
                print(
                    f"Warning: Missing MBM file for {glacier_name} ({year}).")
                mbm_mb.append(np.nan)
            else:
                ds = xr.open_dataset(file_path)
                mbm_mb.append(ds["pred_masked"].mean().values)

            # Get GLAMOS Balance for the year, or return NaN if missing
            glamos_mb.append(GLAMOS_glwmb["GLAMOS Balance"].get(year, np.nan))

        # Store mean values, ignoring NaNs
        mbm_mb_mean.append(np.nanmean(mbm_mb))
        glamos_mb_mean.append(np.nanmean(glamos_mb))
        geodetic_mb.append(geoMBs[periods.index(period)])
        target_period.append(period)

    # Only plot if more than one valid measurement exists
    valid_measurements = np.isfinite(geodetic_mb) & np.isfinite(
        mbm_mb_mean) & np.isfinite(glamos_mb_mean)

    if np.sum(valid_measurements) > 1:
        fig = plotGeodeticMB(geodetic_mb, mbm_mb_mean, glamos_mb_mean,
                             glacier_name, color_xgb, color_tim)

        # Ensure layout is adjusted before saving
        fig.tight_layout()
        fig.savefig(os.path.join(path_figure, f"{glacier_name}.png"))
        plt.close()

    else:
        print(
            f"Skipping {glacier_name}: Not enough valid measurements for plotting."
        )

#### One plot:

In [ ]:
test_glaciers = [
    'tortin', 'plattalva', 'sanktanna', 'schwarzberg', 'hohlaub', 'pizol',
    'corvatsch', 'tsanfleuron', 'forno'
]

# Storage lists for results
mbm_mb_mean, glamos_mb_mean, geodetic_mb = [], [], []
gl, period_len, gl_type, area = [], [], [], []

for glacier_name in tqdm(glacier_list_geod, desc="Processing glaciers"):

    # Check if GLAMOS file exists before proceeding
    glamos_file = os.path.join(path_SMB_GLAMOS_csv, "fix",
                               f"{glacier_name}_fix.csv")
    if not os.path.exists(glamos_file):
        continue  # Skip glacier if no GLAMOS data available

    # Load GLAMOS data
    GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name)
    if GLAMOS_glwmb is None:
        print(f"Skipping {glacier_name}: Failed to load GLAMOS data.")
        continue

    # Get glacier-specific periods and geodetic MBs
    periods = periods_per_glacier.get(glacier_name, [])
    geoMBs = geoMB_per_glacier.get(glacier_name, [])

    if not periods or not geoMBs:
        print(
            f"Skipping {glacier_name}: No geodetic mass balance data available."
        )
        continue

    # Path to model predictions
    folder_path = os.path.join(PATH_PREDICTIONS, glacier_name)

    for period in periods:
        mbm_mb, glamos_mb = [], []

        # Check if any required year is missing in the dataset
        if check_missing_years(folder_path, glacier_name, period):
            continue

        for year in range(period[0], period[1] + 1):
            file_path = os.path.join(folder_path,
                                     f"{glacier_name}_{year}_annual.nc")

            # Check if the NetCDF file exists before opening
            if not os.path.exists(file_path):
                print(
                    f"Warning: Missing MBM file for {glacier_name} ({year}).")
                mbm_mb.append(np.nan)
            else:
                ds = xr.open_dataset(file_path)
                mbm_mb.append(ds["pred_masked"].mean().values)

            # Get GLAMOS Balance for the year, handling missing years
            glamos_mb.append(GLAMOS_glwmb["GLAMOS Balance"].get(year, np.nan))

        # Store mean values, ignoring NaNs
        mbm_mb_mean.append(np.nanmean(mbm_mb))
        glamos_mb_mean.append(np.nanmean(glamos_mb))
        geodetic_mb.append(geoMBs[periods.index(period)])
        gl.append(glacier_name)
        gl_type.append(glacier_name in test_glaciers)
        period_len.append(period[1] - period[0])
        area.append(gl_area.get(glacier_name, np.nan))  # Handle missing areas

# Create DataFrame
df_all = pd.DataFrame({
    "MBM MB": mbm_mb_mean,
    "GLAMOS MB": glamos_mb_mean,
    "Geodetic MB": geodetic_mb,
    "GLACIER": gl,
    "Period Length": period_len,
    "Test Glacier": gl_type,
    "Area": area  # in km²
})

# Sort by area for better visualization
df_all.sort_values(by="Area", inplace=True, ascending=True)

In [ ]:
# All glaciers
scatter_geodeticMB(df_all, hue='GLACIER', size=True)

# Plot scatterplot but size is  proportional to glacier area
scatter_geodeticMB(df_all, hue='Test Glacier', size=True)

In [ ]:
# Only longest period per glacier
df_longest = df_all.loc[df_all.groupby('GLACIER')['Period Length'].idxmax()]
df_longest.sort_values(by='Area', inplace=True)
scatter_geodeticMB(df_longest, size=True)
plt.suptitle('Only longest period per glacier')
plt.tight_layout()

## Glacier wide MB:

In [ ]:
# Define glacier name
glacier_name = "silvretta"

# Load GLAMOS data
GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name)

# Check if GLAMOS data is available
if GLAMOS_glwmb is None:
    print(f"Skipping {glacier_name}: No GLAMOS data available.")
else:
    # Define the path to model predictions
    path_results = os.path.join(PATH_PREDICTIONS, glacier_name)

    # Extract available years from NetCDF filenames
    years = sorted([
        int(f.split("_")[1]) for f in os.listdir(path_results)
        if f.endswith("_annual.nc")
    ])

    if not years:
        print(f"Skipping {glacier_name}: No model prediction files found.")
    else:
        # Extract model-predicted mass balance
        pred_gl = []
        for year in years:
            file_path = os.path.join(path_results,
                                     f"{glacier_name}_{year}_annual.nc")
            if not os.path.exists(file_path):
                print(
                    f"Warning: Missing MBM file for {glacier_name} ({year}). Skipping..."
                )
                pred_gl.append(np.nan)
                continue

            ds = xr.open_dataset(file_path)
            pred_gl.append(ds.pred_masked.mean().item())

        # Create DataFrame
        MBM_glwmb = pd.DataFrame(pred_gl, index=years, columns=["MBM Balance"])

        # Merge with GLAMOS data
        MBM_glwmb = MBM_glwmb.join(GLAMOS_glwmb)

        # Drop NaN values to avoid plotting errors
        MBM_glwmb = MBM_glwmb.dropna()

        # Ensure data exists before plotting
        if MBM_glwmb.empty:
            print(f"Skipping {glacier_name}: No valid data to plot.")
        else:
            # Plot the data
            fig, ax = plt.subplots(figsize=(12, 6))
            MBM_glwmb.plot(ax=ax, marker="o", color=[color_xgb, color_tim])
            ax.set_title(f"{glacier_name.capitalize()} Glacier", fontsize=24)
            ax.set_ylabel("Mass Balance [m w.e.]", fontsize=18)
            ax.set_xlabel("Year", fontsize=18)
            ax.grid(True, linestyle="--", linewidth=0.5)
            
            # increase font size of legends
            ax.legend(fontsize=14)

            plt.show()

In [ ]:
# Define output folder
output_folder = "figures/glacierwide/"
emptyfolder(output_folder)  # Clear existing figures

# Process each glacier
for glacier_name in tqdm(glacier_list_geod):
    # Load GLAMOS data
    GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name)

    # Check if GLAMOS data is available
    if GLAMOS_glwmb is None:
        print(f"Skipping {glacier_name}: No GLAMOS data available.")
        continue

    # Define the path to model predictions
    glacier_path = os.path.join(PATH_PREDICTIONS, glacier_name)

    # Check if glacier prediction directory exists
    if not os.path.exists(glacier_path):
        print(f"Skipping {glacier_name}: No prediction data available.")
        continue

    # Extract available years from NetCDF filenames
    try:
        years = sorted([
            int(f.split("_")[1]) for f in os.listdir(glacier_path)
            if f.endswith("_annual.nc") and "_" in f
        ])
    except ValueError:
        print(
            f"Skipping {glacier_name}: Failed to extract years from filenames."
        )
        continue

    if not years:
        print(f"Skipping {glacier_name}: No model prediction files found.")
        continue

    # Extract model-predicted mass balance
    pred_gl = []
    for year in years:
        file_path = os.path.join(glacier_path,
                                 f"{glacier_name}_{year}_annual.nc")

        if not os.path.exists(file_path):
            print(
                f"Warning: Missing MBM file for {glacier_name} ({year}). Skipping..."
            )
            pred_gl.append(np.nan)
            continue

        ds = xr.open_dataset(file_path)
        pred_gl.append(ds.pred_masked.mean().item())

    # Create DataFrame
    glwide_MB = pd.DataFrame(pred_gl, index=years, columns=["MBM Balance"])

    # Merge with GLAMOS data
    glwide_MB = glwide_MB.join(GLAMOS_glwmb)

    # Drop NaN values to avoid plotting errors
    glwide_MB = glwide_MB.dropna()

    # Ensure data exists before plotting
    if glwide_MB.empty:
        print(f"Skipping {glacier_name}: No valid data to plot.")
        continue

    # Plot the data
    fig, ax = plt.subplots(figsize=(12, 6))
    glwide_MB.plot(ax=ax, marker="o", color=[color_xgb, color_tim])
    ax.set_title(f"{glacier_name.capitalize()} Glacier", fontsize=24)
    ax.set_ylabel("Mass Balance [m w.e.]", fontsize=18)
    ax.set_xlabel("Year", fontsize=18)
    ax.grid(True, linestyle="--", linewidth=0.5)
    
    # increase font size of legends
    ax.legend(fontsize=14)

    # Save the figure
    output_path = os.path.join(output_folder, f"{glacier_name}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")

    # Clear the plot to free memory
    plt.close(fig)

print("Processing complete. Figures saved in:", output_folder)

## Distributed GLAMOS:

In [ ]:
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(path_PMB_GLAMOS_csv, "CH_wgms_dataset_all.csv")
if not os.path.exists(stake_file):
    print("Error: Stake data file not found. Exiting.")
else:
    df_stakes = pd.read_csv(stake_file)

In [ ]:
def apply_gaussian_filter(ds, variable_name='pred_masked', sigma: float = 1):
    # Get the DataArray for the specified variable
    data_array = ds[variable_name]

    # Step 1: Create a mask of valid data (non-NaN values)
    mask = ~np.isnan(data_array)

    # Step 2: Replace NaNs with zero (or a suitable neutral value)
    filled_data = data_array.fillna(0)

    # Step 3: Apply Gaussian filter to the filled data
    smoothed_data = gaussian_filter(filled_data, sigma=sigma)

    # Step 4: Restore NaNs to their original locations
    smoothed_data = xr.DataArray(smoothed_data,
                                 dims=data_array.dims,
                                 coords=data_array.coords,
                                 attrs=data_array.attrs).where(
                                     mask)  # Apply the mask to restore NaNs
    ds[variable_name] = smoothed_data
    return ds

In [ ]:
# # Define glacier name
# glacier_name = "aletsch"

# # Get available years from filenames
# years = sorted(
#     map(int, [
#         f.split("_")[1]
#         for f in os.listdir(os.path.join(PATH_PREDICTIONS, glacier_name))
#         if "_" in f and f.endswith("_annual.nc")
#     ]))

# # if not exist create path:
# if not os.path.exists(f"figures/dst_mb/{glacier_name}"):
#     os.makedirs(f"figures/dst_mb/{glacier_name}")
# # else empty:
# else:
#     emptyfolder(f"figures/dst_mb/{glacier_name}")

# # Iterate through each year
# for year in tqdm(years):
#     print(f"Processing: {glacier_name}, Year: {year}")

#     # Construct file path for GLAMOS distributed MB
#     file_name = f"{year}_ann_fix_lv95.grid"
#     grid_file_path = os.path.join(path_distributed_MB_glamos, glacier_name,
#                                   file_name)

#     # if path exists
#     if not os.path.exists(grid_file_path):
#         print(f"Skipping {year}: GLAMOS file missing for {glacier_name}.")
#         continue

#     # Load GLAMOS data
#     metadata, grid_data = load_grid_file(grid_file_path)

#     # Convert to xarray and transform to WGS84
#     ds_glamos = convert_to_xarray_geodata(grid_data, metadata)
#     ds_glamos_wgs84 = transform_xarray_coords_lv95_to_wgs84(ds_glamos)

#     # Load MBM predictions
#     mbm_file_path = os.path.join(PATH_PREDICTIONS, glacier_name,
#                                  f"{glacier_name}_{year}_annual.nc")

#     if not os.path.exists(mbm_file_path):
#         print(f"Skipping {year}: MBM file missing for {glacier_name}.")
#         continue

#     ds_mbm = xr.open_dataset(mbm_file_path)
#     ds_mbm = apply_gaussian_filter(ds_mbm)

#     # Compute color scale limits
#     vmin = min(ds_glamos_wgs84.min().item(), ds_mbm.pred_masked.min().item())
#     vmax = max(ds_glamos_wgs84.max().item(), ds_mbm.pred_masked.max().item())
#     norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

#     print(f"Color scale range: vmin={vmin:.3f}, vmax={vmax:.3f}")

#     # Plot the data
#     fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

#     # GLAMOS Plot
#     ds_glamos_wgs84.plot.imshow(ax=axes[0],
#                                 cmap="coolwarm_r",
#                                 norm=norm,
#                                 cbar_kwargs={"label": "Mass Balance [m w.e.]"})
#     axes[0].set_title("GLAMOS")

#     # MBM Predictions Plot
#     ds_mbm.pred_masked.plot(ax=axes[1],
#                             cmap="coolwarm_r",
#                             norm=norm,
#                             cbar_kwargs={"label": "Mass Balance [m w.e.]"})
#     axes[1].set_title("MBM")

#     # Set figure title
#     plt.suptitle(f"{glacier_name.capitalize()}: Summer {year}")

#     # Remove grid
#     for ax in axes:
#         ax.grid(False)

#     # Adjust layout and show plot
#     plt.tight_layout()

#     # Save the figure
#     output_path = os.path.join(f"figures/dst_mb/{glacier_name}",
#                                f"{glacier_name}_{year}.png")
#     plt.savefig(output_path, dpi=300, bbox_inches="tight")

#     plt.close()

### Validate against stake data:

In [ ]:
def plot_mass_balance(glacier_name, year, df_stakes,
                      path_distributed_MB_glamos, PATH_PREDICTIONS):
    """
    Plots the annual and winter mass balance for a given glacier and year,
    comparing GLAMOS distributed MB with model predictions.

    Parameters:
    -----------
    glacier_name : str
        Name of the glacier.
    year : int
        Year for which the mass balance should be plotted.
    df_stakes : pd.DataFrame
        DataFrame containing all stake measurements.
    path_distributed_MB_glamos : str
        Path to the GLAMOS distributed mass balance files.
    PATH_PREDICTIONS : str
        Path to the MBM model predictions.
    """

    print(f"Processing: {glacier_name}, Year: {year}")

    # Extract stake data for this glacier and year
    stakes_data = df_stakes[(df_stakes.GLACIER == glacier_name)
                            & (df_stakes.YEAR == year)]
    stakes_data_ann = stakes_data[stakes_data.PERIOD == "annual"].copy()
    stakes_data_win = stakes_data[stakes_data.PERIOD == "winter"].copy()

    # Construct file paths
    file_ann = f"{year}_ann_fix_lv95.grid"
    file_win = f"{year}_win_fix_lv95.grid"
    grid_path_ann = os.path.join(path_distributed_MB_glamos, glacier_name,
                                 file_ann)
    grid_path_win = os.path.join(path_distributed_MB_glamos, glacier_name,
                                 file_win)

    # Load GLAMOS data (Annual)
    # if exists
    if not os.path.exists(grid_path_ann):
        print(f"Skipping {year}: GLAMOS file missing for {glacier_name}.")
        return None
    metadata_ann, grid_data_ann = load_grid_file(grid_path_ann)
    ds_glamos_ann = convert_to_xarray_geodata(grid_data_ann, metadata_ann)
    ds_glamos_wgs84_ann = transform_xarray_coords_lv95_to_wgs84(ds_glamos_ann)

    # Load GLAMOS data (Winter)
    try:
        metadata_win, grid_data_win = load_grid_file(grid_path_win)
        ds_glamos_win = convert_to_xarray_geodata(grid_data_win, metadata_win)
        ds_glamos_wgs84_win = transform_xarray_coords_lv95_to_wgs84(
            ds_glamos_win)
    except FileNotFoundError:
        print(
            f"Warning: Winter grid data missing for {glacier_name} ({year}).")
        ds_glamos_wgs84_win = None

    # Load MBM predictions (Annual)
    mbm_file_ann = os.path.join(PATH_PREDICTIONS, glacier_name,
                                f"{glacier_name}_{year}_annual.nc")
    ds_mbm_ann = xr.open_dataset(mbm_file_ann)
    ds_mbm_ann = apply_gaussian_filter(ds_mbm_ann)

    # Load MBM predictions (Winter)
    mbm_file_win = os.path.join(PATH_PREDICTIONS, glacier_name,
                                f"{glacier_name}_{year}_winter.nc")
    ds_mbm_win = xr.open_dataset(mbm_file_win)
    ds_mbm_win = apply_gaussian_filter(ds_mbm_win)

    # Ensure correct coordinate names
    lon_name = "lon" if "lon" in ds_mbm_ann.coords else "longitude"
    lat_name = "lat" if "lat" in ds_mbm_ann.coords else "latitude"

    # Function to extract mass balance for each stake
    def get_predicted_mb(row, ds):
        try:
            return ds.sel(
                {
                    lon_name: row.POINT_LON,
                    lat_name: row.POINT_LAT
                },
                method="nearest").pred_masked.item()  # Convert to scalar
        except Exception:
            print(
                f"Warning: Stake at ({row.POINT_LON}, {row.POINT_LAT}) is out of bounds."
            )
            return np.nan

    def get_predicted_mb_glamos(row, ds):
        try:
            return ds.sel({
                lon_name: row.POINT_LON,
                lat_name: row.POINT_LAT
            },
                          method="nearest").item()  # Convert to scalar
        except Exception:
            print(
                f"Warning: Stake at ({row.POINT_LON}, {row.POINT_LAT}) is out of bounds."
            )
            return np.nan

    # Apply the function correctly using lambda
    stakes_data_ann["Predicted_MB"] = stakes_data_ann.apply(
        lambda row: get_predicted_mb(row, ds_mbm_ann), axis=1)
    stakes_data_ann.dropna(subset=["Predicted_MB"], inplace=True)
    stakes_data_ann["GLAMOS_MB"] = stakes_data_ann.apply(
        lambda row: get_predicted_mb_glamos(row, ds_glamos_wgs84_ann), axis=1)
    stakes_data_ann.dropna(subset=["GLAMOS_MB"], inplace=True)

    # Same for winter
    stakes_data_win["Predicted_MB"] = stakes_data_win.apply(
        lambda row: get_predicted_mb(row, ds_mbm_win), axis=1)
    stakes_data_win.dropna(subset=["Predicted_MB"], inplace=True)
    stakes_data_win["GLAMOS_MB"] = stakes_data_win.apply(
        lambda row: get_predicted_mb_glamos(row, ds_glamos_wgs84_win), axis=1)
    stakes_data_win.dropna(subset=["GLAMOS_MB"], inplace=True)

    # Compute color scale limits (Annual)
    vmin_ann = min(ds_glamos_wgs84_ann.min().item(),
                   ds_mbm_ann.pred_masked.min().item())
    vmax_ann = max(ds_glamos_wgs84_ann.max().item(),
                   ds_mbm_ann.pred_masked.max().item())

    # Compute color scale limits (Winter)
    vmin_win = min(ds_glamos_wgs84_win.min().item(),
                   ds_mbm_win.pred_masked.min().item())
    vmax_win = max(ds_glamos_wgs84_win.max().item(),
                   ds_mbm_win.pred_masked.max().item())

    print(
        f"Color scale range (Annual): vmin={vmin_ann:.3f}, vmax={vmax_ann:.3f}"
    )
    print(
        f"Color scale range (Winter): vmin={vmin_win:.3f}, vmax={vmax_win:.3f}"
    )

    if vmin_ann < 0 and vmax_ann > 0:
        norm_ann = mcolors.TwoSlopeNorm(vmin=vmin_ann,
                                        vcenter=0,
                                        vmax=-vmin_ann)
        cmap_ann = "coolwarm_r"
    elif vmin_ann < 0 and vmax_ann < 0:
        norm_ann = mcolors.Normalize(vmin=vmin_ann, vmax=vmax_ann)
        cmap_ann = "Reds"
    else:
        norm_ann = mcolors.Normalize(vmin=vmin_ann, vmax=vmax_ann)
        cmap_ann = "Blues"

    if vmin_win < 0:
        norm_win = mcolors.TwoSlopeNorm(vmin=-vmax_win,
                                        vcenter=0,
                                        vmax=vmax_win)
        cmap_win = "coolwarm_r"
    else:
        norm_win = mcolors.Normalize(vmin=vmin_win, vmax=vmax_win)
        cmap_win = "Blues"

    # Create figure with 2 rows (Annual & Winter)
    fig, axes = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

    # Annual GLAMOS Plot
    ds_glamos_wgs84_ann.plot.imshow(
        ax=axes[0, 0],
        cmap=cmap_ann,
        norm=norm_ann,
        cbar_kwargs={"label": "Mass Balance [m w.e.]"})
    axes[0, 0].set_title("GLAMOS (Annual)")
    sns.scatterplot(data=stakes_data_ann,
                    x="POINT_LON",
                    y="POINT_LAT",
                    hue="POINT_BALANCE",
                    palette=cmap_ann,
                    hue_norm=norm_ann,
                    ax=axes[0, 0],
                    s=25,
                    legend=False)
    
    
    # add rmse if available
    if not stakes_data_ann.empty:
        rmse = mean_squared_error(stakes_data_ann.POINT_BALANCE,
                                  stakes_data_ann.GLAMOS_MB,
                                  squared=False)
        axes[0, 0].text(0.05,
                       0.1,
                       f"RMSE: {rmse:.2f}",
                       transform=axes[0, 0].transAxes,
                       ha='left',
                       va='top',
                       fontsize=18)
    
    # Annual MBM Predictions Plot
    ds_mbm_ann.pred_masked.plot.imshow(
        ax=axes[0, 1],
        cmap=cmap_ann,
        norm=norm_ann,
        cbar_kwargs={"label": "Mass Balance [m w.e.]"})
    axes[0, 1].set_title("MBM (Annual)")

    # Add Annual Stake Coordinates
    sns.scatterplot(data=stakes_data_ann,
                    x="POINT_LON",
                    y="POINT_LAT",
                    hue="POINT_BALANCE",
                    palette=cmap_ann,
                    hue_norm=norm_ann,
                    ax=axes[0, 1],
                    s=25,
                    legend=False)
    
    # add rmse
    if not stakes_data_ann.empty:
        rmse = mean_squared_error(stakes_data_ann.POINT_BALANCE,
                              stakes_data_ann.Predicted_MB,
                              squared=False)
        axes[0, 1].text(0.05,
                   0.1,
                   f"RMSE: {rmse:.2f}",
                   transform=axes[0, 1].transAxes,
                   ha='left',
                   va='top',
                   fontsize=18)

    # Winter GLAMOS & MBM Plots
    ds_glamos_wgs84_win.plot.imshow(
        ax=axes[1, 0],
        cmap=cmap_win,
        norm=norm_win,
        cbar_kwargs={"label": "Mass Balance [m w.e.]"})
    axes[1, 0].set_title("GLAMOS (Winter)")
    sns.scatterplot(data=stakes_data_win,
                    x="POINT_LON",
                    y="POINT_LAT",
                    hue="POINT_BALANCE",
                    palette=cmap_win,
                    hue_norm=norm_win,
                    ax=axes[1, 0],
                    s=25,
                    legend=False)
    
    # add rmse
    if not stakes_data_win.empty:
        rmse = mean_squared_error(stakes_data_win.POINT_BALANCE,
                              stakes_data_win.GLAMOS_MB,
                              squared=False)
        axes[1, 0].text(0.05,
                   0.1,
                   f"RMSE: {rmse:.2f}",
                   transform=axes[1, 0].transAxes,
                   ha='left',
                   va='top',
                   fontsize=18)

    # Winter MBM Predictions Plot
    ds_mbm_win.pred_masked.plot.imshow(
        ax=axes[1, 1],
        cmap=cmap_win,
        norm=norm_win,
        cbar_kwargs={"label": "Mass Balance [m w.e.]"})
    axes[1, 1].set_title("MBM (Winter)")
    sns.scatterplot(data=stakes_data_win,
                    x="POINT_LON",
                    y="POINT_LAT",
                    hue="POINT_BALANCE",
                    palette=cmap_win,
                    hue_norm=norm_win,
                    ax=axes[1, 1],
                    s=25,
                    legend=False)
    
    # add rmse
    if not stakes_data_win.empty:
        rmse = mean_squared_error(stakes_data_win.POINT_BALANCE,
                              stakes_data_win.Predicted_MB,
                              squared=False)
        axes[1, 1].text(0.05,
                   0.1,
                   f"RMSE: {rmse:.2f}",
                   transform=axes[1, 1].transAxes,
                   ha='left',
                   va='top',
                   fontsize=18)

    plt.suptitle(f"{glacier_name.capitalize()}: Mass Balance {year}")
    
    plt.tight_layout()

    return fig

In [ ]:
glacier_name = "rhone"
year = 2022
fig = plot_mass_balance(glacier_name, year, df_stakes,
                              path_distributed_MB_glamos, PATH_PREDICTIONS)
plt.show()

In [ ]:
# Define glacier name
glacier_name = "tsanfleuron"

# Get available years from filenames
years = sorted(
    map(int, [
        f.split("_")[1]
        for f in os.listdir(os.path.join(PATH_PREDICTIONS, glacier_name))
        if "_" in f and f.endswith("_annual.nc")
    ]))

# if not exist create path:
if not os.path.exists(f"figures/dst_mb/{glacier_name}"):
    os.makedirs(f"figures/dst_mb/{glacier_name}")
# else empty:
else:
    emptyfolder(f"figures/dst_mb/{glacier_name}")

# Iterate through each year
for year in tqdm(years):
    print(f"Processing: {glacier_name}, Year: {year}")

    fig = plot_mass_balance(glacier_name, year, df_stakes,
                              path_distributed_MB_glamos, PATH_PREDICTIONS)

    # save figure
    output_path = os.path.join(f"figures/dst_mb/{glacier_name}",
                               f"{glacier_name}_{year}.png")
    # if fig not none:
    if fig:
        fig.savefig(output_path, dpi=300, bbox_inches="tight")
        plt.close()

## Input grids:

In [ ]:
glacier_name = 'aletsch'
year = 1952
month = 'jul'

# Feature columns:
feature_columns = [
    'ELEVATION_DIFFERENCE'
] + list(vois_climate) + list(vois_topographical) + ['pcsr']
all_columns = feature_columns + cfg.fieldsNotFeatures

glacier_file = os.path.join(path_glacier_grid_glamos,glacier_name,
                            f"{glacier_name}_grid_{year}.csv")

df_grid_monthly = pd.read_csv(glacier_file)

# Correct climate grids:
# Take the biggest grid cell value for each month
for voi in [
        't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10',
        'ALTITUDE_CLIMATE'
]:
    df_grid_monthly = correct_for_biggest_grid(
        df_grid_monthly, group_columns=["YEAR", "MONTHS"], value_column=voi)

# New elevation difference:
df_grid_monthly['ELEVATION_DIFFERENCE'] = df_grid_monthly[
    "POINT_ELEVATION"] - df_grid_monthly["ALTITUDE_CLIMATE"]

# apply T & P correction
# Apply temperature gradient correction
temp_grad = -6.5 / 1000
dpdz = 1.5 / 10000
c_prec = 1.434,
t_off = 0.617

# Apply temperature correction factor
df_grid_monthly['t2m_corr'] = df_grid_monthly['t2m'] + (
    df_grid_monthly['ELEVATION_DIFFERENCE'] * temp_grad)
df_grid_monthly['t2m_corr'] += t_off

# Apply elevation correction factor
df_grid_monthly['tp_corr'] = df_grid_monthly['tp'] * c_prec
df_grid_monthly['tp_corr'] += df_grid_monthly['tp_corr'] * (
    df_grid_monthly['ELEVATION_DIFFERENCE'] * dpdz)

# Rename aspect and slope to sgi
df_grid_monthly.rename(columns={
    'aspect': 'aspect_sgi',
    'slope': 'slope_sgi'
},
                       inplace=True)
df_grid_monthly['POINT_ELEVATION'] = df_grid_monthly['topo']
df_grid_monthly.drop_duplicates(inplace=True)  # remove duplicates
df_grid_monthly = df_grid_monthly[all_columns]

df_grid_month = df_grid_monthly[df_grid_monthly.MONTHS == month]

In [ ]:
# Define features to plot
v_plot = ["tp_corr", "t2m_corr"] + vois_topographical + ['slhf', 'sshf']

# Ensure the grid layout fits the number of plots dynamically
n_rows = 3
n_cols = min(3, len(v_plot))  # Ensure no more than 4 columns

# Create figure and axes
fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 10))

# Flatten the axes array for easier indexing
axs = axs.flatten()

# Plot each feature
for i, voi in enumerate(v_plot):
    sns.scatterplot(data=df_grid_month,
                    x="POINT_LON",
                    y="POINT_LAT",
                    hue=voi,
                    ax=axs[i],
                    palette="twilight_shifted",
                    s=2)
    axs[i].set_title(voi)
    axs[i].set_ylabel("Lat")
    axs[i].set_xlabel("Lon")
    axs[i].legend().remove()
    axs[i].grid(False)

# Remove unused subplots if any
for j in range(i + 1, len(axs)):
    fig.delaxes(axs[j])

# Adjust layout for clarity
plt.tight_layout()
plt.show()

## Stake coverage:

In [ ]:
# Define year and month for analysis
year = 2018
month = "jul"

# Ensure the output folder is empty
output_folder = "figures/stake_coverage/"
emptyfolder(output_folder)

# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(path_PMB_GLAMOS_csv, "CH_wgms_dataset_all.csv")
if not os.path.exists(stake_file):
    print("Error: Stake data file not found. Exiting.")
else:
    df_stakes = pd.read_csv(stake_file)

# Iterate through each glacier
for glacier_name in tqdm(glacier_list_geod, desc="Processing glaciers"):
    # Construct glacier grid file path
    glacier_file = os.path.join(path_glacier_grid_sgi,
                                f"{glacier_name}_grid_{year}.csv")

    if not os.path.exists(glacier_file):
        print(f"Skipping {glacier_name}: Grid file not found.")
        continue  # Skip if glacier grid file is missing

    # Load and preprocess glacier grid data
    df_grid_monthly = pd.read_csv(glacier_file)
    df_grid_monthly = correct_vars_grid(df_grid_monthly)

    # Rename aspect and slope to sgi
    df_grid_monthly.rename(columns={
        "aspect": "aspect_sgi",
        "slope": "slope_sgi"
    },
                           inplace=True)
    df_grid_monthly["POINT_ELEVATION"] = df_grid_monthly["topo"]
    df_grid_monthly.drop_duplicates(inplace=True)  # Remove duplicates

    # Filter dataset for selected month
    df_grid_month = df_grid_monthly[df_grid_monthly.MONTHS == month]

    # Filter stakes for this glacier
    df_stakes_glacier = df_stakes[df_stakes["GLACIER"] == glacier_name].copy()

    # If no stake data available for this glacier, continue
    if df_stakes_glacier.empty:
        print(f"Skipping {glacier_name}: No stake data available.")
        continue

    # Convert to GeoDataFrame for geospatial visualization
    df_stakes_glacier["geometry"] = gpd.points_from_xy(
        df_stakes_glacier.POINT_LON, df_stakes_glacier.POINT_LAT)
    gdf_stakes = gpd.GeoDataFrame(df_stakes_glacier, crs="EPSG:4326")

    # Load SGI grid
    sgi_file = os.path.join(path_SGI_topo,
                            f"xr_masked_grids/{glacier_name}.nc")

    if not os.path.exists(sgi_file):
        print(f"Skipping {glacier_name}: SGI NetCDF file not found.")
        continue

    sgi_grid = xr.open_dataset(sgi_file)

    # Create figure and plot
    fig, ax = plt.subplots(figsize=(5, 5))

    # Plot glacier mask
    sgi_grid.glacier_mask.plot(cmap="binary", ax=ax, alpha=0.5)

    # Scatter plot of the monthly glacier grid with elevation difference
    scatter = sns.scatterplot(data=df_grid_month,
                              x="POINT_LON",
                              y="POINT_LAT",
                              hue="ELEVATION_DIFFERENCE",
                              ax=ax,
                              s=5,
                              palette="twilight_shifted",
                              legend=True)

    # Plot stake locations
    gdf_stakes.plot(ax=ax,
                    color=color_tim,
                    edgecolor="black",
                    markersize=20,
                    label="Stakes")

    # Remove automatic Seaborn legend
    ax.legend().remove()

    # Set title
    ax.set_title(f"{glacier_name}")

    # Save figure
    output_path = os.path.join(output_folder, f"{glacier_name}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")

    # Clear the plot to free memory
    plt.close(fig)

print("Processing complete. Figures saved to:", output_folder)

## S2 Sentinel data:

### Get S2 rasters:
Get the names of all satellite nc files and classifiy them per month and hydrological year.

In [ ]:
# Sentinel-2
path_S2 = '../../../data/Sentinel/'
src_crs = 'EPSG:4326'  # Original CRS (lat/lon) wgs84

# Organize the rasters by hydrological year
satellite_years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]

rasters_S2 = organize_rasters_by_hydro_year(path_S2, satellite_years)

# Print the organized rasters
for hydro_year, months in rasters_S2.items():
    print(f"-----------------\nHydrological Year: {hydro_year}")
    for month, files in months.items():
        print(f"  {month}: {files}")

### Create gdf for S2 rasters:
Create panadas geodataframe for each satellite raster.

In [ ]:
satellite_glaciers = [
    'adler', 'aletsch', 'allalin', 'basodino', 'claridenL', 'claridenU',
    'findelen', 'gries', 'hohlaub', 'limmern', 'oberaar', 'plattalva', 'rhone',
    'sanktanna', 'schwarzbach', 'schwarzberg'
]

processed_gl = os.listdir(PATH_PREDICTIONS)

# list of gl in processed_gl that are also in satellite_glaciers
satellite_glaciers = [gl for gl in processed_gl if gl in satellite_glaciers]
satellite_glaciers

## SGI rasters:

### S2 rasters over glaciers:

In [ ]:
RUN = False
if RUN:
    # emptyfolder(os.path.join(path_S2, 'perglacier/'))
    # For each year and month where there is S2 data
    for hydro_year, months in tqdm(rasters_S2.items(),
                                   position=0,
                                   desc='Hydrological Years'):
        for month, files in tqdm(months.items(),
                                 position=1,
                                 leave=False,
                                 desc='Months'):

            # More than one if S2 A and B flew in the same month
            for raster_name in files:
                # Create raster for that year & month:
                file_date = datetime.strptime(
                    raster_name.split('_')[3][:8],
                    "%Y%m%d")  # Extract the 8-digit date (YYYYMMDD)

                # Path to the raster for that year & month:
                raster_path = os.path.join(path_S2, file_date.strftime('%Y'),
                                           raster_name)

                # Creates a geodataframe from the .tif raster
                # This is the longest part of the code
                geo_data_S2 = mbm.GeoData(pd.DataFrame)
                gdf_S2 = geo_data_S2.raster_to_gpd(raster_path)
                geo_data_S2.set_gdf(gdf_S2)

                # For each glacier, clip the S2 raster to the glacier extent
                for glacierName in tqdm(satellite_glaciers,
                                        position=2,
                                        leave=False,
                                        desc='Glaciers'):

                    # Load MB predictions for that year and month
                    path_nc_wgs84 = PATH_PREDICTIONS + f"{glacierName}/"
                    filename_nc = f"{glacierName}_{hydro_year}_{month_abbr_hydr_full[month]}.nc"

                    # check if file exists:
                    if not os.path.exists(
                            os.path.join(path_nc_wgs84, filename_nc)):
                        continue

                    # Open xarray dataset and set to class:
                    geoData_gl = mbm.GeoData(pd.DataFrame)
                    geoData_gl.set_ds_latlon(filename_nc, path_nc_wgs84)

                    # Clip the S2 dataframe to the glacier extent
                    # and resample it to the glacier resolution
                    gdf_raster_res = mbm.GeoData.resample_satellite_to_glacier(
                        geoData_gl.gdf, gdf_S2)

                    # In case the glacier is outside of the bounds of the raster
                    if gdf_raster_res is 0:
                        continue
                    # In case the raster is empty where the glacier is
                    elif gdf_raster_res is 1:
                        continue
                    else:
                        pass

                    # Check the percentage of cloud cover
                    cloud_cover_glacier = gdf_raster_res.classes[
                        gdf_raster_res.classes ==
                        5].count() / gdf_raster_res.classes.count()

                    # If the cloud cover is too high, skip the glacier
                    if cloud_cover_glacier > 0.5:
                        continue

                    # Save the glacier raster:
                    S2_gl_name = '{}_{}.geojson'.format(
                        glacierName, file_date.strftime('%Y_%m_%d'))
                    S2_gl_path = os.path.join(path_S2, 'perglacier',
                                              S2_gl_name)
                    gdf_raster_res.to_file(S2_gl_path, driver="GeoJSON")

## Snow cover:

In [ ]:
RUN = True
if RUN:

    # first delete results file
    if os.path.exists(f'results/snow_cover.csv'):
        os.remove(f'results/snow_cover.csv')

    # Add header
    with open(f'results/snow_cover.csv', 'w') as f:
        f.write(
            "year,month,monthNb,raster_date,glacier_name,snow_cover_S2,snow_cover_glacier\n"
        )

    # All rasters resampled over glaciers:
    rasters_resampled = os.listdir(os.path.join(path_S2, 'perglacier'))

    for raster_res in tqdm(rasters_resampled):
        # Extract glacier name
        glacierName = raster_res.split('_')[0]

        # Extract date from satellite raster
        match = re.search(r"(\d{4})_(\d{2})_(\d{2})", raster_res)
        year, month, day = match.groups()
        date_str = match.group(1) + '-' + match.group(2) + '-' + match.group(3)
        raster_date = datetime.strptime(date_str, "%Y-%m-%d")

        # Find closest hydrological year and month
        closest_month, hydro_year = get_hydro_year_and_month(raster_date)
        monthNb = month_abbr_hydr_full[closest_month]

        if hydro_year > 2021:
            continue

        # Read satellite raster over glacier (previously resampled)
        gdf_S2_res = gpd.read_file(
            os.path.join(path_S2, 'perglacier', raster_res))

        # Calculate percentage of snow cover (class 1)
        snow_cover_S2 = IceSnowCover(gdf_S2_res, gdf_S2_res)

        # Load MB predictions for that year and month
        path_nc_wgs84 = PATH_PREDICTIONS + f"{glacierName}/"
        filename_nc = f"{glacierName}_{hydro_year}_{monthNb}.nc"

        geoData_gl = mbm.GeoData(pd.DataFrame)
        geoData_gl.set_ds_latlon(filename_nc, path_nc_wgs84)
        geoData_gl.classify_snow_cover(tol=0.1)

        snow_cover_glacier = IceSnowCover(geoData_gl.gdf, gdf_S2_res)

        # Save the results
        with open(f'results/snow_cover.csv', 'a') as f:
            f.write(
                f"{hydro_year},{closest_month},{monthNb},{date_str},{glacierName},{snow_cover_S2},{snow_cover_glacier}\n"
            )

In [ ]:
df = pd.read_csv(f'results/snow_cover.csv').sort_values(by=['year', 'month'],
                                                        ascending=True)
# remove october and september
df = df[~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter(df, add_corr=False)
plt.show()

In [ ]:
df_gl = df[(df.glacier_name == 'aletsch') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

df_gl = df[(df.glacier_name == 'gries') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

df_gl = df[(df.glacier_name == 'rhone') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

df_gl = df[(df.glacier_name == 'adler') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

plt.show()

In [ ]:
# Loop through the raster files and plot geoplots
gl = 'aletsch'
yr = 2021
rasters_gl = [
    f for f in os.listdir(os.path.join(path_S2, 'perglacier'))
    if (gl in f) and (str(yr) in f)
]
rasters_gl.sort()
for raster_res in rasters_gl:
    plot_snow_cover_geoplots(raster_res, path_S2, month_abbr_hydr_full)


#### Sensitivity of tolerance:

In [ ]:
RUN = False

if RUN:
    # first delete results file
    if os.path.exists(f'results/snow_cover_tol.csv'):
        os.remove(f'results/snow_cover_tol.csv')

    # Add header
    with open(f'results/snow_cover_tol.csv', 'w') as f:
        f.write(
            "tol,year,month,monthNb,raster_date,glacier_name,snow_cover_S2,snow_cover_glacier,snow_cover_glacier_corr\n"
        )

    # All rasters resampled over glaciers:
    rasters_resampled = [
        f for f in os.listdir(os.path.join(path_S2, 'perglacier'))
    ]

    for tol in tqdm(np.arange(0, 1, 0.1), position=0, desc='Tolerance'):
        for raster_res in tqdm(rasters_resampled):
            # Extract glacier name
            glacierName = raster_res.split('_')[0]

            # Extract date from satellite raster
            match = re.search(r"(\d{4})_(\d{2})_(\d{2})", raster_res)
            year, month, day = match.groups()
            date_str = match.group(1) + '-' + match.group(
                2) + '-' + match.group(3)
            raster_date = datetime.strptime(date_str, "%Y-%m-%d")

            # Find closest hydrological year and month
            closest_month, hydro_year = get_hydro_year_and_month(raster_date)
            monthNb = month_abbr_hydr_full_hydr[closest_month]

            if hydro_year > 2021:
                continue

            # Read satellite raster over glacier (previously resampled)
            gdf_S2_res = gpd.read_file(
                os.path.join(path_S2, 'perglacier', raster_res))

            # Calculate percentage of snow cover (class 1)
            snow_cover_S2 = IceSnowCover(gdf_S2_res, gdf_S2_res)

            # Load MB predictions for that year and month
            path_nc_wgs84 = f"results/nc/var_normal/{glacierName}/wgs84/"
            path_nc_wgs84_corr = f"results/nc/var_corr/{glacierName}/wgs84/"
            filename_nc = f"{glacierName}_{hydro_year}_{monthNb}.nc"

            gdf_glacier = ClassSnowCover(path_nc_wgs84, filename_nc, tol)
            snow_cover_glacier = IceSnowCover(gdf_glacier, gdf_S2_res)

            # Corrected T and P
            gdf_glacier_corr = ClassSnowCover(path_nc_wgs84_corr, filename_nc,
                                              tol)
            snow_cover_glacier_corr = IceSnowCover(gdf_glacier_corr,
                                                   gdf_S2_res)

            # Save the results
            with open(f'results/snow_cover_tol.csv', 'a') as f:
                f.write(
                    f"{tol},{hydro_year},{closest_month},{monthNb},{date_str},{glacierName},{snow_cover_S2},{snow_cover_glacier},{snow_cover_glacier_corr}\n"
                )

df = pd.read_csv(f'results/snow_cover_tol.csv').sort_values(
    by=['year', 'month'], ascending=True)
df_gl = df[~df.month.isin(['oct', 'sep'])]

for tol in df_gl.tol.unique():
    df_gl_tol = df_gl[df_gl.tol == tol]
    # fig, axs = plot_snow_cover_scatter_combined(df_gl_tol)
    fig, axs = plot_snow_cover_scatter(df_gl_tol, add_corr=False)
    plt.suptitle(f'Tolerance: {np.round(tol, 4)}', fontsize=14)

In [ ]:
# Initialize a dictionary to store the results for each tolerance
tol_results = {}
month_labels = ['April', 'May', 'June', 'July', 'August']
# Iterate over each tolerance value
for tol in df_gl.tol.unique():
    df_gl_tol = df_gl[df_gl.tol == tol]

    # Get sorted unique months
    months = np.sort(df_gl_tol['monthNb'].unique())

    # Lists to store metrics for each month
    r2 = []
    rmse = []

    # Loop over each month
    for monthNb in months:
        df_month = df_gl_tol[df_gl_tol['monthNb'] == monthNb]

        # Calculate R^2
        r2.append(
            np.corrcoef(df_month['snow_cover_S2'],
                        df_month['snow_cover_glacier'])[0, 1]**2)

        # Calculate MSE
        rmse.append(
            mean_squared_error(df_month['snow_cover_glacier'],
                               df_month['snow_cover_S2'],
                               squared=False))

    # Store the results in the dictionary
    tol_results[np.round(tol, 4)] = {
        'months': month_labels,
        'r2': r2,
        'rmse': rmse
    }

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True)

# Create a colormap
cmap = plt.cm.Blues
norm = plt.Normalize(vmin=min(df_gl.tol.unique()),
                     vmax=max(df_gl.tol.unique()))

# Plot R^2 values for each tolerance
for tol, results in tol_results.items():
    color = cmap(norm(tol))
    axes[0].plot(results['months'],
                 results['r2'],
                 marker='o',
                 label=f'Tol {tol} m',
                 color=color)

# Plot MSE values for each tolerance
for tol, results in tol_results.items():
    color = cmap(norm(tol))
    axes[1].plot(results['months'],
                 results['rmse'],
                 marker='o',
                 label=f'Tol {tol} m',
                 color=color)

# Customize the plots
axes[0].set_title('R^2 by Month and Tolerance')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('R^2')
axes[0].legend()

axes[1].set_title('MSE by Month and Tolerance')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('MSE')
axes[1].legend()

plt.tight_layout()
plt.show()

### Sensitivity of prec & temp correction:

In [ ]:
scores = pd.read_csv('results/sensitivity_scores.csv')
scores

In [ ]:
def extract_c_prec_t_off(filename):
    """
    Extract c_prec and t_off values from a given filename.

    Parameters:
        filename (str): The string containing the values to extract.

    Returns:
        tuple: A tuple (c_prec, t_off) with the extracted values as floats.
    """
    # Define the regular expression pattern
    #pattern = r"_(\d+\.\d+)_(-?\d+\.\d+)\.nc$"
    pattern = r"_(\d+\.\d+)_(-?\d+)\.nc$"

    # Search for the pattern in the filename
    match = re.search(pattern, filename)

    if match:
        # Extract the matched groups and convert to floats
        c_prec = float(match.group(1))
        t_off = float(match.group(2))
        return c_prec, t_off
    else:
        raise ValueError("Filename does not match the expected pattern")


# Example usage
filename = "hohlaub_2015_1_1.6_0.3333.nc"
c_prec, t_off = extract_c_prec_t_off(filename)
print(f"c_prec: {c_prec}, t_off: {t_off}")

In [ ]:
def filter_files_by_pattern(file_list,
                            glacierName=None,
                            hydro_year=None,
                            monthNb=None):
    """
    Filter files starting with the pattern "{glacierName}_{hydro_year}_{monthNb}".

    Parameters:
        file_list (list): List of filenames to filter.
        glacierName (str, optional): The name of the glacier to match. Defaults to None (any glacier).
        hydro_year (int, optional): The hydrological year to match. Defaults to None (any year).
        monthNb (int, optional): The month number to match. Defaults to None (any month).

    Returns:
        list: A list of filenames that match the pattern.
    """
    # Build the regular expression pattern based on the provided attributes
    glacier_pattern = glacierName if glacierName else r"[a-zA-Z0-9]+"
    year_pattern = str(hydro_year) if hydro_year else r"\d{4}"
    month_pattern = str(monthNb) if monthNb else r"\d{1,2}"

    pattern = rf"^{glacier_pattern}_{year_pattern}_{month_pattern}.*"

    # Filter the files that match the pattern
    matching_files = [
        filename for filename in file_list if re.match(pattern, filename)
    ]

    return matching_files

In [ ]:
files = os.listdir('results/nc/sensitivity/adler/wgs84')
matching_files = filter_files_by_pattern(files,
                                         glacierName='adler',
                                         hydro_year=2015,
                                         monthNb=9)
matching_files

In [ ]:
satellite_glaciers = os.listdir('results/nc/sensitivity/')
RUN = True
if RUN:
    # first delete results file
    if os.path.exists(f'results/snow_cover_corr.csv'):
        os.remove(f'results/snow_cover_corr.csv')

    # Add header
    with open(f'results/snow_cover_corr.csv', 'w') as f:
        f.write(
            "c_prec,t_off,year,month,monthNb,raster_date,glacier_name,snow_cover_S2,snow_cover_glacier\n"
        )

    # All rasters resampled over glaciers:
    rasters_resampled = os.listdir(os.path.join(path_S2, 'perglacier'))

    for raster_res in tqdm(rasters_resampled):
        # Extract glacier name
        glacierName = raster_res.split('_')[0]

        # Extract date from satellite raster
        match = re.search(r"(\d{4})_(\d{2})_(\d{2})", raster_res)
        year, month, day = match.groups()
        date_str = match.group(1) + '-' + match.group(2) + '-' + match.group(3)
        raster_date = datetime.strptime(date_str, "%Y-%m-%d")

        # Find closest hydrological year and month
        closest_month, hydro_year = get_hydro_year_and_month(raster_date)
        monthNb = month_abbr_hydr_full_hydr[closest_month]

        if hydro_year > 2021:
            continue

        # Read satellite raster over glacier (previously resampled)
        gdf_S2_res = gpd.read_file(
            os.path.join(path_S2, 'perglacier', raster_res))

        # Calculate percentage of snow cover (class 1)
        snow_cover_S2 = IceSnowCover(gdf_S2_res, gdf_S2_res)

        # Load MB predictions for that year and month
        path_nc_wgs84 = f"results/nc/sensitivity/{glacierName}/wgs84/"

        files_list = os.listdir(path_nc_wgs84)
        matching_files_list = filter_files_by_pattern(files_list,
                                                      glacierName=glacierName,
                                                      hydro_year=hydro_year,
                                                      monthNb=monthNb)
        for filename_nc in matching_files_list:
            c_prec, t_off = extract_c_prec_t_off(filename_nc)

            gdf_glacier = ClassSnowCover(path_nc_wgs84, filename_nc)
            snow_cover_glacier = IceSnowCover(gdf_glacier, gdf_S2_res)

            # Save the results
            with open(f'results/snow_cover_corr.csv', 'a') as f:
                f.write(
                    f"{c_prec},{t_off},{hydro_year},{closest_month},{monthNb},{date_str},{glacierName},{snow_cover_S2},{snow_cover_glacier}\n"
                )

In [ ]:
# read results
df = pd.read_csv(f'results/snow_cover_corr.csv').sort_values(
    by=['year', 'month'], ascending=True)
df_gl = df[~df.month.isin(['oct', 'sep'])]

factors = df_gl[['c_prec', 't_off']].drop_duplicates().sort_values(by='c_prec')

for i, row in factors.iterrows():
    c_prec, t_off = row['c_prec'], row['t_off']
    df_gl_c_prec = df_gl[(df_gl.c_prec == c_prec) & (df_gl.t_off == t_off)]
    fig, axs = plot_snow_cover_scatter(df_gl_c_prec, add_corr=False)
    plt.suptitle(f'c_prec: {c_prec}, t_off: {t_off}', fontsize=14)

## Snowline:

In [ ]:
# Loop through the raster files and plot geoplots
gl = 'aletsch'
yr = 2021
rasters_gl = [
    f for f in os.listdir(os.path.join(path_S2, 'perglacier'))
    if (gl in f) and (str(yr) in f)
]
rasters_gl.sort()
for raster_res in rasters_gl:
    plot_snow_cover_geoplots(raster_res,
                             path_S2,
                             month_abbr_hydr_full_hydr,
                             add_snowline=True,
                             band_size=50,
                             percentage_threshold=50)


In [ ]:
# Loop through the raster files and plot geoplots
gl = 'rhone'
yr = 2021
rasters_gl = [
    f for f in os.listdir(os.path.join(path_S2, 'perglacier'))
    if (gl in f) and (str(yr) in f)
]
rasters_gl.sort()
for raster_res in rasters_gl:
    plot_snow_cover_geoplots(raster_res,
                             path_S2,
                             month_abbr_hydr_full_hydr,
                             add_snowline=True,
                             band_size=50,
                             percentage_threshold=50)


## Missed glaciers:

#### Problem 1: Glacier is in regions where raster is NaN

In [ ]:
year = 2015
glacierName = 'taelliboden'
filename_nc = f"{glacierName}_{year}.nc"
path_nc_wgs84 = f"results/nc/var_normal/{glacierName}/wgs84/"
path_nc_wgs84_corr = f"results/nc/var_corr/{glacierName}/wgs84/"

gdf_raster = createRaster(raster_a_2015)

# Calculate snow cover for glacier
gdf_glacier, snow_cover_glacier, ice_cover_glacier = snowCover(
    path_nc_wgs84, filename_nc)
# Corrected for temperature & precipitation
gdf_glacier, snow_cover_glacier_corr, ice_cover_glacier_corr = snowCover(
    path_nc_wgs84_corr, filename_nc)

# Clip the raster to the glacier extent and resample it to the glacier resolution
# gdf_raster_res = resampleRaster(gdf_glacier, gdf_raster)

bounding_box = gdf_glacier.total_bounds  # [minx, miny, maxx, maxy]
raster_bounds = gdf_raster.total_bounds  # [minx, miny, maxx, maxy]

# Check if glacier bounds are within raster bounds
if not (bounding_box[0] >= raster_bounds[0]
        and  # minx of glacier >= minx of raster
        bounding_box[1] >= raster_bounds[1]
        and  # miny of glacier >= miny of raster
        bounding_box[2] <= raster_bounds[2]
        and  # maxx of glacier <= maxx of raster
        bounding_box[3]
        <= raster_bounds[3]  # maxy of glacier <= maxy of raster
        ):
    print(f"Glacier {glacierName} is out of bounds")

bbox_polygon = box(*bounding_box)

# The raster might have no data (NaN values) in the region of the glacier:
bounding_box = [7.8, 45.95854232, 8, 46.1]
bbox_polygon = box(*bounding_box)
gfd_res = gdf_raster[gdf_raster.intersects(bbox_polygon)]
ax = gfd_res.plot(color='blue', alpha=0.5)
gdf_glacier.plot(ax=ax, color='red')

## Transform to tif rasters for QGIS:

### One year:

In [ ]:
glacierName = 'aletsch'
year = 2021

filename_nc = f"{glacierName}_{year}.nc"
path_nc_lv95 = f"results/nc/{glacierName}/lv95/"
path_nc_wgs84 = f"results/nc/{glacierName}/wgs84/"

filename_tif = f"{glacierName}_{year}.tif"
path_tif_wgs84 = f"results/tif/{glacierName}/wgs84/"
path_tif_lv95 = f"results/tif/{glacierName}/lv95/"

createPath(path_tif_lv95)
createPath(path_tif_wgs84)

In [ ]:
gdf, gdf_class, raster_data, extent = TransformToRaster(
    filename_nc, filename_tif, path_nc_wgs84, path_tif_wgs84, path_tif_lv95)

In [ ]:
API_KEY = "000378bd-b0f0-46e2-a46d-f2165b0c6c02"
provider = cx.providers.Stadia.StamenTerrain(api_key=API_KEY)
provider["url"] = provider["url"] + f"?api_key={API_KEY}"

vmin, vmax = gdf.data.min(), gdf.data.max()
norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
gdf_clean = gdf.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    cmap="RdBu",  # Color map suitable for glacier data
    norm=norm,
    legend=True,  # Display a legend
    ax=axs[0],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[0], crs=gdf.crs, source=provider)

gdf_clean = gdf_class.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    legend=True,  # Display a legend
    ax=axs[1],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[1], crs=gdf.crs, source=provider)
plt.tight_layout()

In [ ]:
vmin, vmax = gdf.data.min(), gdf.data.max()

# Calculate the relative position of 0
relative_position = (0 - vmin) / (vmax - vmin) * 100

# Print the result
print(f"The relative position of 0 is {relative_position:.2f}%")

### All years:

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
for year in years:
    print(year)
    for month in month_abbr_hydr_full_hydr:
        monthNb = month_abbr_hydr_full_hydr[month]

        filename_nc = f"{glacierName}_{year}_{monthNb}.nc"
        path_nc_lv95 = f"results/nc/{glacierName}/lv95/"
        path_nc_wgs84 = f"results/nc/{glacierName}/wgs84/"

        filename_tif = f"{glacierName}_{year}_{monthNb}.tif"
        path_tif_wgs84 = f"results/tif/{glacierName}/wgs84/"
        path_tif_lv95 = f"results/tif/{glacierName}/lv95/"  # normally EPSG Code: 2056

        gdf, gdf_class, raster_data, extent = TransformToRaster(
            filename_nc, filename_tif, path_nc_wgs84, path_tif_wgs84,
            path_tif_lv95)

In [ ]:
API_KEY = "000378bd-b0f0-46e2-a46d-f2165b0c6c02"
provider = cx.providers.Stadia.StamenTerrain(api_key=API_KEY)
provider["url"] = provider["url"] + f"?api_key={API_KEY}"

vmin, vmax = gdf.data.min(), gdf.data.max()
norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig, axs = plt.subplots(1, 2, figsize=(5, 5))
gdf_clean = gdf.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    cmap="RdBu",  # Color map suitable for glacier data
    norm=norm,
    legend=True,  # Display a legend
    ax=axs[0],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[0], crs=gdf.crs, source=provider)

gdf_clean = gdf_class.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    legend=True,  # Display a legend
    ax=axs[1],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[1], crs=gdf.crs, source=provider)

### Step by step example of one file:

### Step 1: Open the NetCDF file


In [ ]:
ds_xy = xr.open_dataset(path_nc_lv95 + filename_nc)
ds_latlon = xr.open_dataset(path_nc_wgs84 + filename_nc)

# Smoothing
ds_xy_g = GaussianFilter(ds_xy)
ds_latlon_g = GaussianFilter(ds_latlon)

# Show effet of Smoothing:
vmin, vmax = np.min([
    ds_xy.pred_masked.min().values,
    ds_xy_g.pred_masked.min()
]), np.max([ds_xy.pred_masked.max().values,
            ds_xy_g.pred_masked.max()])
max_abs_value = max(abs(vmin), abs(vmax))
norm = mcolors.TwoSlopeNorm(vmin=-max_abs_value, vcenter=0, vmax=max_abs_value)

fig, axs = plt.subplots(1, 2, figsize=(20, 10))
ds_xy.pred_masked.plot.imshow(cmap='RdBu', norm=norm, ax=axs[0])
axs[0].set_title('Original')

# Plot or analyze `smoothed_data` as needed
ds_xy_g.pred_masked.plot.imshow(cmap='RdBu', norm=norm, ax=axs[1])
axs[1].set_title('Gaussian Filter')

# print min and max values
print(ds_xy.pred_masked.min().values, ds_xy.pred_masked.max().values)
print(ds_xy_g.pred_masked.min().values, ds_xy_g.pred_masked.max().values)

### Step 2: transform to geopandas

In [ ]:
gdf, lon, lat = toGeoPandas(ds_latlon_g)

# Reproject to LV95 (EPSG:2056) swiss coordinates
# gdf_lv95 = gdf.to_crs("EPSG:2056")

API_KEY = "000378bd-b0f0-46e2-a46d-f2165b0c6c02"
provider = cx.providers.Stadia.StamenTerrain(api_key=API_KEY)
provider["url"] = provider["url"] + f"?api_key={API_KEY}"

fig, ax = plt.subplots(figsize=(5, 5))
gdf_clean = gdf.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    cmap="Reds",  # Color map suitable for glacier data
    legend=True,  # Display a legend
    ax=ax,
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(ax, crs=gdf.crs, source=provider)

### Step 3: transform to raster

In [ ]:
# Transform to raster from geopandas
raster_data, extent = toRaster(gdf,
                               lon,
                               lat,
                               file_name=path_tif_wgs84 + filename_tif)

# reproject raster to Swiss coordinates (LV95)
reproject_raster_to_lv95(path_tif_wgs84 + filename_tif,
                         path_tif_lv95 + filename_tif)

### Opt step 4: for clariden
Need to merge two rasters

In [ ]:
if 'clariden' in glacierName:
    merge_rasters('results/tif/claridenL_2022_w_lv95.tif',
                  'results/tif/claridenU_2022_w_lv95.tif',
                  'results/tif/clariden_2022_w_lv95.tif')